## Remove Bismillah

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Arabic-Original.csv', header=None, names=['surah_number', 'aayah_number', 'transcription'], delimiter='|')

df.head(10)

In [ ]:
df.shape

In [ ]:
df = df.query('aayah_number != 1')

In [ ]:
df.shape

In [ ]:
df.to_csv('Arabic-Original-no-bismillah.csv', index=False)

## Create transcription dataset file

In [1]:
import pandas as pd
import swifter
import os
import glob
import librosa

f:\Productivity\Github-Repos\whisper_transcription\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('Arabic-Original-no-bismillah.csv')
df.head(10)

,surah_number,aayah_number,transcription
0,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
1,1,3,الرَّحْمَٰنِ الرَّحِيمِ
2,1,4,مَالِكِ يَوْمِ الدِّينِ
3,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
4,1,6,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ
5,1,7,صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...
6,2,2,ذَٰلِكَ الْكِتَابُ لَا رَيْبَ ۛ فِيهِ ۛ هُدًى ...
7,2,3,الَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ...
8,2,4,وَالَّذِينَ يُؤْمِنُونَ بِمَا أُنْزِلَ إِلَيْك...
9,2,5,أُولَٰئِكَ عَلَىٰ هُدًى مِنْ رَبِّهِمْ ۖ وَأُو...


In [3]:
df.shape

(6122, 3)

In [4]:
base_dir = 'F:\\Productivity\\Work\\Furqan\\data\\bismillah'

In [5]:
imam_items = os.listdir(base_dir)
imam_dirs = [item for item in imam_items if os.path.isdir(os.path.join(base_dir, item))]
imam_dirs

['Abdullaah_3awwaad_Al-Juhaynee_128kbps',
 'Abdullah_Basfar_192kbps',
 'Abdullah_Matroud_128kbps',
 'AbdulSamad_64kbps_QuranExplorer.Com',
 'Abdul_Basit_Mujawwad_128kbps',
 'Abdul_Basit_Murattal_192kbps',
 'Abdurrahmaan_As-Sudais_192kbps',
 'Abu_Bakr_Ash-Shaatree_128kbps',
 'Ahmed_ibn_Ali_al-Ajamy_128kbps_ketaballah.net',
 'ahmed_ibn_ali_al_ajamy_128kbps',
 'Ahmed_Neana_128kbps',
 'Akram_AlAlaqimy_128kbps',
 'Alafasy_128kbps',
 'Ali_Hajjaj_AlSuesy_128kbps',
 'Ayman_Sowaid_64kbps',
 'Fares_Abbad_64kbps',
 'Ghamadi_40kbps',
 'Hani_Rifai_192kbps',
 'Hudhaify_128kbps',
 'Husary_128kbps',
 'Husary_128kbps_Mujawwad',
 'Husary_Muallim_128kbps',
 'Khaalid_Abdullaah_al-Qahtaanee_192kbps',
 'MaherAlMuaiqly128kbps',
 'Menshawi_16kbps',
 'Minshawy_Murattal_128kbps',
 'Mohammad_al_Tablaway_128kbps',
 'Muhammad_AbdulKareem_128kbps',
 'Muhammad_Ayyoub_128kbps',
 'Muhammad_Jibreel_128kbps',
 'Muhsin_Al_Qasim_192kbps',
 'Nabil_Rifa3i_48kbps',
 'Nasser_Alqatami_128kbps',
 'Sahl_Yassin_128kbps',
 'Salaah

In [6]:
def get_aayah_df(imam_dir):
    aayah_files = glob.glob(os.path.join(base_dir, imam_dir, '*.mp3'))
    df = pd.DataFrame(aayah_files, columns=['file_path'])
    df = pd.DataFrame(df['file_path'].str.split('\\', expand=True)[7])
    df = pd.DataFrame(df[7].str.replace('.mp3', '').str.split('_', expand=True),).rename(columns={0: 'surah_number', 1: 'aayah_number'})
    df['file_path'] = aayah_files 
    df['surah_number'] = df['surah_number'].astype(int)
    df['aayah_number'] = df['aayah_number'].astype(int)
    return df

def get_voice_length(file_path):
    length = None
    try:
        y, sr = librosa.load(file_path)
        length =  librosa.get_duration(y=y, sr=sr)
    except:
        print(f"Error in file: {file_path}")
    return length

In [7]:
combined_df = None

for i in range(len(imam_dirs)):
    df_imam = get_aayah_df(imam_dirs[i])

    df_imam = pd.merge(df, df_imam, on=['surah_number', 'aayah_number'])

    if combined_df is None:
        combined_df = df_imam
        
    else:
        combined_df = pd.concat([combined_df, df_imam], ignore_index=True)

In [8]:
combined_df.head(10)

,surah_number,aayah_number,transcription,file_path
0,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,F:\Productivity\Work\Furqan\data\bismillah\Abd...
1,1,3,الرَّحْمَٰنِ الرَّحِيمِ,F:\Productivity\Work\Furqan\data\bismillah\Abd...
2,1,4,مَالِكِ يَوْمِ الدِّينِ,F:\Productivity\Work\Furqan\data\bismillah\Abd...
3,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,F:\Productivity\Work\Furqan\data\bismillah\Abd...
4,1,6,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ,F:\Productivity\Work\Furqan\data\bismillah\Abd...
5,1,7,صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...,F:\Productivity\Work\Furqan\data\bismillah\Abd...
6,2,2,ذَٰلِكَ الْكِتَابُ لَا رَيْبَ ۛ فِيهِ ۛ هُدًى ...,F:\Productivity\Work\Furqan\data\bismillah\Abd...
7,2,3,الَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ...,F:\Productivity\Work\Furqan\data\bismillah\Abd...
8,2,4,وَالَّذِينَ يُؤْمِنُونَ بِمَا أُنْزِلَ إِلَيْك...,F:\Productivity\Work\Furqan\data\bismillah\Abd...
9,2,5,أُولَٰئِكَ عَلَىٰ هُدًى مِنْ رَبِّهِمْ ۖ وَأُو...,F:\Productivity\Work\Furqan\data\bismillah\Abd...


In [9]:
combined_df.shape

(147804, 4)

In [10]:
final_df = None

In [11]:
combined_df['voice_length'] = None

In [12]:
combined_df

,surah_number,aayah_number,transcription,file_path,voice_length
0,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,None
1,1,3,الرَّحْمَٰنِ الرَّحِيمِ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,None
2,1,4,مَالِكِ يَوْمِ الدِّينِ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,None
3,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,None
4,1,6,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,None
...,...,...,...,...,...
147799,21,41,وَلَقَدِ اسْتُهْزِئَ بِرُسُلٍ مِنْ قَبْلِكَ فَ...,F:\Productivity\Work\Furqan\data\bismillah\Yas...,None
147800,21,42,قُلْ مَنْ يَكْلَؤُكُمْ بِاللَّيْلِ وَالنَّهَار...,F:\Productivity\Work\Furqan\data\bismillah\Yas...,None
147801,21,43,أَمْ لَهُمْ آلِهَةٌ تَمْنَعُهُمْ مِنْ دُونِنَا...,F:\Productivity\Work\Furqan\data\bismillah\Yas...,None
147802,21,44,بَلْ مَتَّعْنَا هَٰؤُلَاءِ وَآبَاءَهُمْ حَتَّى...,F:\Productivity\Work\Furqan\data\bismillah\Yas...,None


In [ ]:
# for i in range(len(combined_df)):
#     print(f"processing {i}/{len(combined_df)}")
#     file_path = combined_df.iloc[i]['file_path']
#     try:
#         voice_length = get_voice_length(file_path)
#         combined_df.at[i, 'voice_length'] = voice_length
#     except Exception as e:
#         print(f"Error in file: {file_path}")
#         continue

In [13]:
combined_df['voice_length'] = combined_df['file_path'].swifter.apply(get_voice_length)

Pandas Apply:   3%|▎         | 4536/147804 [47:21<15:21:18,  2.59it/s] f:\Productivity\Github-Repos\whisper_transcription\.venv\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


Error in file: F:\Productivity\Work\Furqan\data\bismillah\Abdullah_Basfar_192kbps\7_110.mp3


Pandas Apply:   4%|▎         | 5405/147804 [1:00:16<34:06:18,  1.16it/s]

Error in file: F:\Productivity\Work\Furqan\data\bismillah\Abdullah_Basfar_192kbps\16_41.mp3


Pandas Apply:   4%|▍         | 6517/147804 [1:12:40<7:49:28,  5.02it/s]  

...

Pandas Apply:  97%|█████████▋| 143373/147804 [20:36:27<36:31,  2.02it/s]  

Error in file: F:\Productivity\Work\Furqan\data\bismillah\Salah_Al_Budair_128kbps\41_15.mp3


Pandas Apply:  98%|█████████▊| 144606/147804 [20:43:14<21:11,  2.51it/s]  

Error in file: F:\Productivity\Work\Furqan\data\bismillah\Salah_Al_Budair_128kbps\74_2.mp3


Pandas Apply: 100%|██████████| 147804/147804 [21:12:51<00:00,  1.94it/s]  


In [14]:
combined_df.head(10)

,surah_number,aayah_number,transcription,file_path,voice_length
0,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,4.912109
1,1,3,الرَّحْمَٰنِ الرَّحِيمِ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,3.736599
2,1,4,مَالِكِ يَوْمِ الدِّينِ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,4.337415
3,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,5.539048
4,1,6,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ,F:\Productivity\Work\Furqan\data\bismillah\Abd...,4.494150
5,1,7,صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...,F:\Productivity\Work\Furqan\data\bismillah\Abd...,14.028844
6,2,2,ذَٰلِكَ الْكِتَابُ لَا رَيْبَ ۛ فِيهِ ۛ هُدًى ...,F:\Productivity\Work\Furqan\data\bismillah\Abd...,6.165986
7,2,3,الَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ...,F:\Productivity\Work\Furqan\data\bismillah\Abd...,8.908844
8,2,4,وَالَّذِينَ يُؤْمِنُونَ بِمَا أُنْزِلَ إِلَيْك...,F:\Productivity\Work\Furqan\data\bismillah\Abd...,10.868027
9,2,5,أُولَٰئِكَ عَلَىٰ هُدًى مِنْ رَبِّهِمْ ۖ وَأُو...,F:\Productivity\Work\Furqan\data\bismillah\Abd...,8.360272


In [15]:
combined_df.to_csv('Arabic-Original-no-bismillah-audio-length.csv', index=False)

f:\Productivity\Github-Repos\whisper_transcription\.venv\lib\site-packages\pandas\core\internals\blocks.py:2540: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [16]:
combined_df.isna().sum()

surah_number       0
aayah_number       0
transcription      0
file_path          0
voice_length     254
dtype: int64

In [19]:
combined_df.dropna().to_csv('Arabic-Original-no-bismillah-audio-length-no-null.csv', index=False)